In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib.request
import time
import os

In [2]:
pq = [
    'Economy',
    'People',
    'Environment',
    'Mobility',
    'Living',
    'Governance',
    'Technology',
    'Infrastructure',
    'Energy',
    'Pandemic Resiliency'
]

In [3]:
# 페이지를 아래로 스크롤하는 함수
def scroll_down(n):
    for i in range(n):
        time.sleep(3)
        # 페이지 맨 아래로 스크롤
        driver.find_element(By.XPATH, '//body').send_keys(Keys.END)
        time.sleep(3)
        try:
            # '더보기' 버튼이 보이면 클릭
            load_more_button = driver.find_element(By.XPATH, '//*[@id="islmp"]/div/div/div/div/div[1]/div[2]/div[2]/input')
            if load_more_button.is_displayed():
                load_more_button.click()
        except:
            pass
        time.sleep(3)

In [12]:
for i in pq:
    if __name__ == "__main__":
        query = i       # 검색어 
        image_cnt = 10 # 수집할 이미지 개수
        save_dir = "saved_image"  # 저장할 디렉토리 이름
        os.makedirs(save_dir, exist_ok=True)  # 디렉토리 생성 (이미 존재하면 무시)
        os.chdir(save_dir)  # 작업 디렉토리 변경

        driver = webdriver.Chrome()  # Chrome 웹 드라이버 실행
        URL = 'https://www.google.com/search?tbm=isch&q='
        driver.get(URL + query)  # 검색어를 포함한 URL로 이동

        scroll_down(1)  # 페이지 스크롤 함수 호출

        # 이미지 정보 추출
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        image_info_list = soup.find_all('img', class_='rg_i')
        image_and_name_list = []

        print('=== 이미지 수집 시작 ===')

        downlaod_cnt = 0
        for i in range(len(image_info_list)):
            if i == image_cnt:
                break
            if 'data-src' in image_info_list[i].attrs:
                save_image = image_info_list[i]['data-src']

                image_path = os.path.join(query.replace(' ', '_') + '_' + str(downlaod_cnt) + '.jpg')
                image_and_name_list.append((save_image, image_path))
                downlaod_cnt += 1

        # 이미지 다운로드
        for i in range(len(image_and_name_list)):
            urllib.request.urlretrieve(image_and_name_list[i][0], image_and_name_list[i][1])

        print('=== 이미지 수집 종료 ===')
        driver.close()  # 브라우저 닫기

NameError: name 'pq' is not defined

In [5]:
query = "Economy"       # 검색어 
image_cnt = 10 # 수집할 이미지 개수
save_dir = "saved_image"  # 저장할 디렉토리 이름
os.makedirs(save_dir, exist_ok=True)  # 디렉토리 생성 (이미 존재하면 무시)
os.chdir(save_dir)  # 작업 디렉토리 변경

driver = webdriver.Chrome()  # Chrome 웹 드라이버 실행
URL = 'https://www.google.com/search?tbm=isch&q='
driver.get(URL + query)  # 검색어를 포함한 URL로 이동

scroll_down(1)  # 페이지 스크롤 함수 호출

# 이미지 정보 추출
soup = BeautifulSoup(driver.page_source, 'html.parser')
image_info_list = soup.find_all('img', class_='rg_i')
image_and_name_list = []

print(image_info_list)

[]


In [ ]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import json

chrome_options = webdriver.ChromeOptions()
chrome_options.binary = '/Users/chemseok/miniforge/envs/epicone/bin/chromedriver'
query = '루피'

driver = webdriver.Chrome()
driver.get(f'https://www.google.com/imghp')

search_bar = driver.find_element(By.NAME,"q")
search_bar.send_keys(query)
search_bar.submit()

PAUSE_TIME = 2
last_hegiht = driver.execute_script("return document.body.scrollHeight")
new_height = 0

while True :

    driver.execute_script("window.scrollBy(0,5000)")

    time.sleep(PAUSE_TIME)

    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height - last_hegiht > 0 :
        last_hegiht = new_height
        continue
    else :
        break

img_elements = driver.find_elements(By.CSS_SELECTOR,".rg_i")
img_rst = []

    for idx, img in enumerate(img_elements) :
        print(f"{query} : {idx+1}/{len(img_elements)} proceed...")

    try :

    img.click()

    time.sleep(PAUSE_TIME)

    img_element = driver.find_element(By.XPATH,'//*[@id="Sva75c"]/div[2]/div/div[2]/div[2]/div[2]/c-wiz/div[2]/div[1]/div[1]/div[2]/div/a/img')

    img_src = img_element.get_attribute('src')

    img_alt = img_element.get_attribute('alt')

    img_rst.append({

    'alt' : img_alt,

    'src' : img_src

    })

    except :

    pass

    driver.close()

    결과 확인
    print(img_rst)

In [7]:
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image
import io
import time
# 크롬드라이버 위치 설정
DRIVER_PATH = '/usr/local/bin/chromedriver'
# 크롬 드라이버 옵션 설정
listm=['가수 백예린']
for i in listm:
  options = Options()
  options.add_argument('--headless')  # 창 안띄우기
  options.add_argument('--no-sandbox')  # 리눅스 환경에서 필요한 옵션
  options.add_argument('--disable-dev-shm-usage')  # 리눅스 환경에서 필요한 옵션
  # 검색어 설정
  search_name = i
  # 검색어를 이용한 구글 이미지 검색 url
  url = f'https://www.google.com/search?q={search_name}&source=lnms&tbm=isch'
  # 크롬 드라이버 실행
  service = Service(DRIVER_PATH)
  driver = webdriver.Chrome()
  # url 접속
  driver.get(url)
  # 페이지 로드를 위한 대기 시간
  time.sleep(2)
  # 이미지 로딩을 위한 스크롤 다운
  last_height = driver.execute_script("return document.body.scrollHeight")
  while True:
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      time.sleep(1)
      new_height = driver.execute_script("return document.body.scrollHeight")
      if new_height == last_height:
          break
      last_height = new_height
  # 이미지 링크 추출
  soup = BeautifulSoup(driver.page_source, 'html.parser')
  img_tags = soup.find_all('img')
  urls = []
  for img in img_tags:
      try:
          url = img['src']
          if 'http' in url:
              urls.append(url)
      except:
          pass
  # 이미지 다운로드
  os.makedirs(f'./images{search_name}', exist_ok=True)
  count = 0
  for url in urls:
      try:
          response = requests.get(url, stream=True)
          # 이미지 사이즈 확인
          img = Image.open(io.BytesIO(response.content))
          width, height = img.size
          if width >= 20 and height >= 20:
              file_name = f'./images{search_name}/{count}.jpg'
              with open(file_name, 'wb') as out_file:
                  out_file.write(response.content)
              print(f'{file_name} saved')
              count += 1
              if count == 200:
                  break
      except:
          pass
  # 크롬 드라이버 종료
  driver.quit()

./images가수 백예린/0.jpg saved
./images가수 백예린/1.jpg saved
./images가수 백예린/2.jpg saved
./images가수 백예린/3.jpg saved
./images가수 백예린/4.jpg saved
./images가수 백예린/5.jpg saved
./images가수 백예린/6.jpg saved
./images가수 백예린/7.jpg saved
./images가수 백예린/8.jpg saved
./images가수 백예린/9.jpg saved
./images가수 백예린/10.jpg saved
./images가수 백예린/11.jpg saved
./images가수 백예린/12.jpg saved
./images가수 백예린/13.jpg saved
./images가수 백예린/14.jpg saved
./images가수 백예린/15.jpg saved
./images가수 백예린/16.jpg saved
./images가수 백예린/17.jpg saved
./images가수 백예린/18.jpg saved
./images가수 백예린/19.jpg saved
./images가수 백예린/20.jpg saved
./images가수 백예린/21.jpg saved
./images가수 백예린/22.jpg saved
./images가수 백예린/23.jpg saved
./images가수 백예린/24.jpg saved
./images가수 백예린/25.jpg saved
./images가수 백예린/26.jpg saved
./images가수 백예린/27.jpg saved
./images가수 백예린/28.jpg saved
./images가수 백예린/29.jpg saved
./images가수 백예린/30.jpg saved
./images가수 백예린/31.jpg saved
./images가수 백예린/32.jpg saved
./images가수 백예린/33.jpg saved
./images가수 백예린/34.jpg saved
./images가수 백예린/35.jpg saved
./